[Homework for AI-For-Beginners course: 05 - Frameworks](https://github.com/microsoft/AI-For-Beginners/blob/main/lessons/3-NeuralNetworks/05-Frameworks/lab/LabFrameworks.ipynb)

In [3]:
# display all outputs from Jupyter notebook cells, not just last.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datasets import load_dataset
import torch
import numpy as np

In [4]:
# load data
dataset = load_dataset("mnist")

In [5]:
# convert into expected torch tensors with image tensor being of shape (batch_size, 784)
def normalize(examples):
    if "image" in examples:
        examples["image"] = [torch.from_numpy(np.array(img).astype("float32") / 255).view(-1).float() for img in examples["image"]]
    if "label" in examples:
        examples["label"] = torch.from_numpy(np.array(examples["label"])).long()
    return examples

dataset = dataset.with_transform(normalize)

In [ ]:
dataset_train, dataset_test = dataset["train"], dataset["test"]
dataset_train.set_format('np')
dataset_test.set_format('np')

x_train = dataset_train['image'].astype("float32") / 255
y_train = keras.utils.to_categorical(dataset_train['label'], 10)

In [6]:
dataset_train, dataset_test = dataset["train"], dataset["test"]
dataset_test_x = torch.stack(dataset_test['image'])
dataset_test_y = dataset_test['label']

In [7]:
dataloader = torch.utils.data.DataLoader(dataset_train, batch_size=32)

In [8]:
def accuracy(dataset_test_x, dataset_test_y):
  _, predicted = torch.max(net(dataset_test_x), 1)
  acc = (predicted == dataset_test_y).float().mean()
  return acc

def train(net, dataloader, dataset_test_x, dataset_test_y, epochs=10, lr=0.05):
  optim = torch.optim.Adam(net.parameters(),lr=lr)
  for ep in range(epochs):
    for batch in dataloader:
      x = batch["image"]
      y = batch["label"]
      z = net(x)
      loss = torch.nn.functional.cross_entropy(z,y)
      optim.zero_grad()
      loss.backward()
      optim.step()
    
    acc = accuracy(dataset_test_x, dataset_test_y)
    print(f"Epoch {ep}: last batch loss = {loss}, val acc = {acc}")

In [9]:
# 1 layer network
net = torch.nn.Sequential(torch.nn.Linear(28*28, 10))

train(net,dataloader,dataset_test_x,dataset_test_y,epochs=2,lr=0.03)

Epoch 0: last batch loss = 0.18834947049617767, val acc = 0.8497999906539917
Epoch 1: last batch loss = 0.05254385992884636, val acc = 0.84579998254776


In [10]:
# 2 layer network
net = torch.nn.Sequential(
    torch.nn.Linear(28*28, 100),  # First layer with 100 neurons
    torch.nn.ReLU(),  # Non-linear activation function
    torch.nn.Linear(100, 10)  # Second layer with 10 neurons (one for each class)
)

train(net,dataloader,dataset_test_x,dataset_test_y,epochs=2,lr=0.03)

Epoch 0: last batch loss = 0.0429919958114624, val acc = 0.8712999820709229
Epoch 1: last batch loss = 0.14745289087295532, val acc = 0.9175000190734863
